In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import pandas as pd
from sklearn import datasets

from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently.presets import DataSummaryPreset #or try DataDriftPreset

In [3]:
from evidently.ui.workspace import CloudWorkspace
import os

In [32]:
api_token = os.getenv('EVIDENTLY_API_TOKEN')
org_id = os.getenv('EVIDENTLY_ORG_ID')
ws = CloudWorkspace(token=api_token, url="https://app.evidently.cloud")

In [35]:
project = ws.create_project("mlapp-demo", org_id=org_id)
project.description = "My project description"
project.save()

# or project = ws.get_project("PROJECT_ID")

In [13]:
import sys
sys.path.append('../src/churn_detection')

In [14]:
from utils.io import split_train_test, read_local_data

X, y = read_local_data(path = '../data/churn_data.csv')

In [18]:
X_train, X_test, y_train, y_test = split_train_test(X, y)

In [19]:
df_train = X_train.copy()
df_train['Churn'] = y_train

df_test = X_test.copy()
df_test['Churn'] = y_test

In [ ]:
from evidently import Dataset
from evidently import DataDefinition

from evidently import BinaryClassification

definition = DataDefinition(
    classification=[BinaryClassification(
        target="Churn", 
        prediction_labels="prediction")])

eval_data_train = Dataset.from_pandas(
    pd.DataFrame(df_train),
    data_definition=definition
)

eval_data_test = Dataset.from_pandas(
    pd.DataFrame(df_test),
    data_definition=definition
)

report = Report([
    DataSummaryPreset() #or try DataDriftPreset()
],
include_tests="True")
my_eval = report.run(eval_data_test, eval_data_train)

In [36]:
ws.add_run(project.id, my_eval, include_data=False)

UUID('0196ede3-80ee-738f-b0d0-e7dbe19f6790')

In [ ]:
df

In [27]:
import json
json.loads(my_eval.json())

{'metrics': [{'id': '00404ffe284d7862baa4095093452630',
   'metric_id': 'RowCount()',
   'value': 5634.0},
  {'id': '91e498263b6502661774b15eb39ea154',
   'metric_id': 'ColumnCount()',
   'value': 20.0},
  {'id': '31b8c78bad2b7108842265cd082d5abf',
   'metric_id': 'ColumnCount(column_type=ColumnType.Numerical)',
   'value': 3.0},
  {'id': 'bb154bb9a843cad0d72fa3dc4983394d',
   'metric_id': 'ColumnCount(column_type=ColumnType.Categorical)',
   'value': 17.0},
  {'id': '012cbfb269361272d4773e1a68559396',
   'metric_id': 'ColumnCount(column_type=ColumnType.Datetime)',
   'value': 0.0},
  {'id': '3f5e595e26bc8e120aadf51967bb0355',
   'metric_id': 'ColumnCount(column_type=ColumnType.Text)',
   'value': 0.0},
  {'id': 'ff1caea5e148b287bd5348d4bb40eccf',
   'metric_id': 'DuplicatedRowCount()',
   'value': 18.0},
  {'id': '02fbcbd017b30a0b075d9d5a72ce2a32',
   'metric_id': 'DuplicatedColumnsCount()',
   'value': 1.0},
  {'id': '4c2efb888b5c5f8afb50895b4b4abcec',
   'metric_id': 'AlmostDuplicat